In [1]:
#run this cell if you're using google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
#path for the file. This is a google drive path, but you can change this path to your own google drive path, or local linux path as per your need

import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [14]:
path = Path('/content/drive/MyDrive/AI\ Stuff/projects/MIT\ CBMM/NSD')
%cd {path}
!ls

/content/drive/MyDrive/AI Stuff/projects/MIT CBMM/NSD
 extract_shared1000.ipynb  'NSD paper.pdf'	       shared1000.mp4
'NSD documentation.gdoc'    nsd_stim_info_merged.csv


In [15]:
img_info = pd.read_csv('nsd_stim_info_merged.csv',drop)
img_info

,Unnamed: 0,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,subject3,subject4,subject5,subject6,subject7,subject8,subject1_rep0,subject1_rep1,subject1_rep2,subject2_rep0,subject2_rep1,subject2_rep2,subject3_rep0,subject3_rep1,subject3_rep2,subject4_rep0,subject4_rep1,subject4_rep2,subject5_rep0,subject5_rep1,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
0,0,532481,val2017,"(0, 0, 0.1671875, 0.1671875)",0.100000,0,False,False,False,0,0,1,0,0,0,0,0,0,0,0,0,0,0,13792,14918,28011,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,245764,val2017,"(0, 0, 0.125, 0.125)",0.000000,1,False,False,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13985,14176,28603,0,0,0
2,2,385029,val2017,"(0, 0, 0.125, 0.125)",0.000000,2,False,False,False,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1042,22634,26547,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,311303,val2017,"(0, 0, 0.16640625, 0.16640625)",0.125000,3,False,False,False,0,0,1,0,0,0,0,0,0,0,0,0,0,0,18180,18551,18638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,393226,val2017,"(0, 0, 0.125, 0.125)",0.133333,4,False,False,False,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13720,22861,23023,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72995,72995,518071,train2017,"(0, 0, 0.125, 0.125)",0.000000,72995,False,False,False,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6083,11650,26531
72996,72996,255930,train2017,"(0, 0, 0.125, 0.125)",0.125000,72996,False,False,False,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10402,10434,10625
72997,72997,255934,train2017,"(0, 0, 0.1, 0.1)",0.000000,72997,False,False,False,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,10007,23243,23363,0,0,0,0,0,0,0,0,0,0,0,0
72998,72998,518080,train2017,"(0.125, 0.125, 0, 0)",0.000000,72998,False,False,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5585,11846,14495,0,0,0


In [17]:
shared_1000 = img_info[img_info['shared1000']==True]
shared_1000

,Unnamed: 0,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,subject3,subject4,subject5,subject6,subject7,subject8,subject1_rep0,subject1_rep1,subject1_rep2,subject2_rep0,subject2_rep1,subject2_rep2,subject3_rep0,subject3_rep1,subject3_rep2,subject4_rep0,subject4_rep1,subject4_rep2,subject5_rep0,subject5_rep1,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
2950,2950,262145,train2017,"(0, 0, 0.16640625, 0.16640625)",0.09375,2950,False,True,True,1,1,1,1,1,1,1,1,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566
2990,2990,262239,train2017,"(0, 0, 0.1671875, 0.1671875)",0.10000,2990,False,True,True,1,1,1,1,1,1,1,1,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711
3049,3049,262414,train2017,"(0, 0, 0.125, 0.125)",0.00000,3049,False,True,True,1,1,1,1,1,1,1,1,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697
3077,3077,524646,train2017,"(0, 0, 0.1671875, 0.1671875)",0.00000,3077,False,True,True,1,1,1,1,1,1,1,1,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537
3146,3146,262690,train2017,"(0, 0, 0.16640625, 0.16640625)",0.00000,3146,False,True,True,1,1,1,1,1,1,1,1,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72312,72312,515508,train2017,"(0, 0, 0.125, 0.125)",0.00000,72312,False,True,True,1,1,1,1,1,1,1,1,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373
72510,72510,254130,train2017,"(0, 0, 0.16640625, 0.16640625)",0.00000,72510,False,True,True,1,1,1,1,1,1,1,1,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310
72605,72605,516634,train2017,"(0.125, 0.125, 0, 0)",0.00000,72605,False,True,True,1,1,1,1,1,1,1,1,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046
72719,72719,304627,train2017,"(0.125, 0.125, 0, 0)",0.00000,72719,False,True,True,1,1,1,1,1,1,1,1,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500


In [19]:
shared_1000.cocoSplit.unique()

array(['train2017'], dtype=object)

We see that these 1000 shared images come from the 2017 version of training images. 

We need annotations to use the coco API. [Here](https://cocodataset.org/#download) are all the download links

In [20]:
#download link
annotations_2017 = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'

!wget {annotations_2017}

--2022-03-02 16:40:07--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.106.196
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.106.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  51.1MB/s    in 4.5s    

2022-03-02 16:40:12 (54.0 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [22]:
!unzip annotations_trainval2017.zip

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


WE also need labels corresponding to each image. This is not provided in this dataset, since classification was not part of the tasks of the experiment. But we can easily get the labels using the COCO api, which can be found [here](https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb). 

In [18]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [23]:
dataDir='.'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [24]:
coco=COCO(annFile)

loading annotations into memory...
Done (t=23.37s)
creating index...
index created!


Our Strategy of labeling each image is not efficient, but it was the only way I found. 

There is a method in the COCO API to get images from category labels, but not the other way round. So the strategy is to list all the images from all the categories and do a search if the image id in the shared 1000 images belongs in the class/superclass. Its not extremely slow - takes about 90 seconds



In [38]:
cats = coco.loadCats(coco.getCatIds())
cats

[{'id': 1, 'name': 'person', 'supercategory': 'person'},
 {'id': 2, 'name': 'bicycle', 'supercategory': 'vehicle'},
 {'id': 3, 'name': 'car', 'supercategory': 'vehicle'},
 {'id': 4, 'name': 'motorcycle', 'supercategory': 'vehicle'},
 {'id': 5, 'name': 'airplane', 'supercategory': 'vehicle'},
 {'id': 6, 'name': 'bus', 'supercategory': 'vehicle'},
 {'id': 7, 'name': 'train', 'supercategory': 'vehicle'},
 {'id': 8, 'name': 'truck', 'supercategory': 'vehicle'},
 {'id': 9, 'name': 'boat', 'supercategory': 'vehicle'},
 {'id': 10, 'name': 'traffic light', 'supercategory': 'outdoor'},
 {'id': 11, 'name': 'fire hydrant', 'supercategory': 'outdoor'},
 {'id': 13, 'name': 'stop sign', 'supercategory': 'outdoor'},
 {'id': 14, 'name': 'parking meter', 'supercategory': 'outdoor'},
 {'id': 15, 'name': 'bench', 'supercategory': 'outdoor'},
 {'id': 16, 'name': 'bird', 'supercategory': 'animal'},
 {'id': 17, 'name': 'cat', 'supercategory': 'animal'},
 {'id': 18, 'name': 'dog', 'supercategory': 'animal'},

In [ ]:
imgIds = coco.getImgIds(catIds=[1]);
imgIds

In [53]:
cocoIds = list(shared_1000.cocoId)
coco_labels = []
coco_lblcategories = []
coco_lblsupercategories = []

for img_id in tqdm(cocoIds):

    for cat in coco.loadCats(coco.getCatIds()):
        imgs_of_the_cat = coco.getImgIds(catIds=[cat['id']])
        if img_id in imgs_of_the_cat:
            coco_labels.append(cat['id'])
            coco_lblcategories.append(cat['name'])
            coco_lblsupercategories.append(cat['supercategory'])
            break
assert len(coco_lblcategories)==len(coco_lblsupercategories)==len(coco_labels) == len(cocoIds)

Adding the labels to the dataframe

In [54]:
shared_1000['coco_label']= coco_labels
shared_1000['category'] = coco_lblcategories
shared_1000['supercategory']= coco_lblsupercategories

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [55]:
shared_1000

,Unnamed: 0,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,subject3,subject4,subject5,subject6,subject7,subject8,subject1_rep0,subject1_rep1,subject1_rep2,subject2_rep0,subject2_rep1,subject2_rep2,subject3_rep0,subject3_rep1,subject3_rep2,subject4_rep0,subject4_rep1,subject4_rep2,subject5_rep0,subject5_rep1,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2,coco_label,category,supercategory
2950,2950,262145,train2017,"(0, 0, 0.16640625, 0.16640625)",0.09375,2950,False,True,True,1,1,1,1,1,1,1,1,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566,1,person,person
2990,2990,262239,train2017,"(0, 0, 0.1671875, 0.1671875)",0.10000,2990,False,True,True,1,1,1,1,1,1,1,1,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711,1,person,person
3049,3049,262414,train2017,"(0, 0, 0.125, 0.125)",0.00000,3049,False,True,True,1,1,1,1,1,1,1,1,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697,1,person,person
3077,3077,524646,train2017,"(0, 0, 0.1671875, 0.1671875)",0.00000,3077,False,True,True,1,1,1,1,1,1,1,1,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537,1,person,person
3146,3146,262690,train2017,"(0, 0, 0.16640625, 0.16640625)",0.00000,3146,False,True,True,1,1,1,1,1,1,1,1,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807,85,clock,indoor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72312,72312,515508,train2017,"(0, 0, 0.125, 0.125)",0.00000,72312,False,True,True,1,1,1,1,1,1,1,1,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,2920,3086,3373,1,person,person
72510,72510,254130,train2017,"(0, 0, 0.16640625, 0.16640625)",0.00000,72510,False,True,True,1,1,1,1,1,1,1,1,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,18619,19110,25310,7,train,vehicle
72605,72605,516634,train2017,"(0.125, 0.125, 0, 0)",0.00000,72605,False,True,True,1,1,1,1,1,1,1,1,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,6893,7036,7046,25,giraffe,animal
72719,72719,304627,train2017,"(0.125, 0.125, 0, 0)",0.00000,72719,False,True,True,1,1,1,1,1,1,1,1,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2926,11601,21500,2,bicycle,vehicle


In [56]:
#Saving the csv file




shared_1000.to_csv('shared_1000_info.csv')